In [ ]:
import csv
import pandas as pd
from collections import Counter
from matplotlib import pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from collections import defaultdict
import math
import re
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn import metrics
import enchant
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
from sklearn.svm import LinearSVC,LinearSVR
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPClassifier,MLPRegressor
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.model_selection import train_test_split
# from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn import tree
# from downstream_models import LogRegClassifier, RandForestClassifier, LinearRegression, RandForestRegressor, MLPRegressorr, MLPClassifierr

import sys

maxintval = sys.maxsize

import warnings
warnings.filterwarnings('ignore')


Hcurstate = 100

In [ ]:
iot = pd.read_csv('dataset/iot.csv')
iot

In [ ]:
numeric_cols = iot[['temp']]
# categ_cols = iot[['Conditions']]
# categ_cols = pd.get_dummies(categ_cols, columns=['Conditions'])
ngram_cols = iot[['room_id/id','noted_date']]

target = iot['out/in']

vectorizer = CountVectorizer(ngram_range=(2,2),analyzer='char')
all_cols = pd.DataFrame()
all_cols = pd.concat([all_cols,numeric_cols], axis=1, sort=False)

for cols in ngram_cols.columns:
    print(cols)
    arr = ngram_cols[cols].astype(str).values
    X = vectorizer.fit_transform(arr)
    tempdf = pd.DataFrame(X.toarray())
    all_cols = pd.concat([all_cols,tempdf], axis=1, sort=False)

all_cols = all_cols.fillna('0')    

In [ ]:
# avgsc_train_lst,avgsc_lst,avgsc_hld_lst = LinearRegression(all_cols,target)
avgsc_train_lst,avgsc_lst,avgsc_hld_lst,cnf_matrix = LogRegClassifier(all_cols,target)

In [ ]:
# avgsc_train_lst,avgsc_lst,avgsc_hld_lst = RandForestRegressor(all_cols,target)
avgsc_train_lst,avgsc_lst,avgsc_hld_lst,cnf_matrix = RandForestClassifier(all_cols,target)

In [ ]:
# avgsc_train_lst,avgsc_lst,avgsc_hld_lst = MLPRegressorr(all_cols,target)
avgsc_train_lst,avgsc_lst,avgsc_hld_lst,cnf_matrix = MLPClassifierr(all_cols,target)

In [ ]:

def LogRegClassifier(data1,y):

    X_train, X_test,y_train,y_test = train_test_split(data1,y, test_size=0.2,random_state=Hcurstate)
    X_train_new = X_train.reset_index(drop=True)
    y_train_new = y_train.reset_index(drop=True)
    
    X_train_new = X_train_new.values
    y_train_new = y_train_new.values

    k = 5
    kf = KFold(n_splits=k,random_state=Hcurstate)
    avg_train_acc,avg_test_acc = 0,0

    val_arr = [0.0001,0.001,0.01,0.1,1,10,100,1000,10000,100000]

    avgsc_lst,avgsc_train_lst,avgsc_hld_lst = [],[],[]
    avgsc,avgsc_train,avgsc_hld = 0,0,0

    for train_index, test_index in kf.split(X_train_new):
        X_train_cur, X_test_cur = X_train_new[train_index], X_train_new[test_index]
        y_train_cur, y_test_cur = y_train_new[train_index], y_train_new[test_index]
        X_train_train, X_val,y_train_train,y_val = train_test_split(X_train_cur,y_train_cur, test_size=0.25,random_state=Hcurstate)

        print(X_train_train.shape)
        print(X_val.shape)
        
        bestPerformingModel = LogisticRegression(penalty='l2',C = 1,random_state=Hcurstate)
        bestscore = 0
        for val in val_arr:
            clf = LogisticRegression(penalty='l2',C = val,random_state=Hcurstate)
            clf.fit(X_train_train, y_train_train)
            sc = clf.score(X_val, y_val)

            if bestscore < sc:
                bestscore = sc
                bestPerformingModel = clf

        bscr_train = bestPerformingModel.score(X_train_cur, y_train_cur)
        bscr = bestPerformingModel.score(X_test_cur, y_test_cur)
        bscr_hld = bestPerformingModel.score(X_test, y_test)

        avgsc_train_lst.append(bscr_train)
        avgsc_lst.append(bscr)
        avgsc_hld_lst.append(bscr_hld)

        avgsc_train = avgsc_train + bscr_train    
        avgsc = avgsc + bscr
        avgsc_hld = avgsc_hld + bscr_hld

        print(bscr_train)
        print(bscr)
        print(bscr_hld)
    
    print('5 fold Train, Validation, and Test Accuracies:')
    print(avgsc_train_lst)
    print(avgsc_lst)
    print(avgsc_hld_lst)

    print('Avg Train, Validation, and Test Accuracies:')    
    print(avgsc_train/k)
    print(avgsc/k)
    print(avgsc_hld/k)
        
    y_pred = bestPerformingModel.predict(X_test)
    cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
    
    return avgsc_train_lst,avgsc_lst,avgsc_hld_lst,cnf_matrix


def RandForestClassifier(data1,y):
    X_train, X_test,y_train,y_test = train_test_split(data1,y, test_size=0.2,random_state=Hcurstate)
    
    X_train_new = X_train.reset_index(drop=True)
    y_train_new = y_train.reset_index(drop=True)
    
    X_train_new = X_train_new.values
    y_train_new = y_train_new.values

    k = 5
    kf = KFold(n_splits=k,random_state=Hcurstate)
    avg_train_acc,avg_test_acc = 0,0

    n_estimators_grid = [5,25,50,75,100,500]
    max_depth_grid = [5,10,25,50,100,500]

    avgsc_lst,avgsc_train_lst,avgsc_hld_lst = [],[],[]
    avgsc,avgsc_train,avgsc_hld = 0,0,0

    for train_index, test_index in kf.split(X_train_new):
        X_train_cur, X_test_cur = X_train_new[train_index], X_train_new[test_index]
        y_train_cur, y_test_cur = y_train_new[train_index], y_train_new[test_index]
        X_train_train, X_val,y_train_train,y_val = train_test_split(X_train_cur,y_train_cur, test_size=0.25,random_state=Hcurstate)

        print(X_train_train.shape)
        print(X_val.shape)            
        
        bestPerformingModel = RandomForestClassifier(n_estimators=10,max_depth=5, random_state=Hcurstate)
        bestscore = 0
        for ne in n_estimators_grid:
            for md in max_depth_grid:
                clf = RandomForestClassifier(n_estimators=ne,max_depth=md, random_state=Hcurstate)
                clf.fit(X_train_train, y_train_train)
                sc = clf.score(X_val, y_val)

                if bestscore < sc:
                    bestscore = sc
                    bestPerformingModel = clf

        bscr_train = bestPerformingModel.score(X_train_cur, y_train_cur)
        bscr = bestPerformingModel.score(X_test_cur, y_test_cur)
        bscr_hld = bestPerformingModel.score(X_test, y_test)

        avgsc_train_lst.append(bscr_train)
        avgsc_lst.append(bscr)
        avgsc_hld_lst.append(bscr_hld)

        avgsc_train = avgsc_train + bscr_train    
        avgsc = avgsc + bscr
        avgsc_hld = avgsc_hld + bscr_hld

        print(bscr_train)
        print(bscr)
        print(bscr_hld)
    
    print('5 fold Train, Validation, and Test Accuracies:')
    print(avgsc_train_lst)
    print(avgsc_lst)
    print(avgsc_hld_lst)
    
    print('Avg Train, Validation, and Test Accuracies:')    
    print(avgsc_train/k)
    print(avgsc/k)
    print(avgsc_hld/k)
        
    y_pred = bestPerformingModel.predict(X_test)
    cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
    
    return avgsc_train_lst,avgsc_lst,avgsc_hld_lst,cnf_matrix


def LinearRegression(data1,y):

    X_train, X_test,y_train,y_test = train_test_split(data1,y, test_size=0.2,random_state=Hcurstate)

    X_train_new = X_train.reset_index(drop=True)
    y_train_new = y_train.reset_index(drop=True)
    
    X_train_new = X_train_new.values
    y_train_new = y_train_new.values

    k = 5
    kf = KFold(n_splits=k,random_state=Hcurstate)
    avg_train_acc,avg_test_acc = 0,0

    val_arr = [0.0001,0.001,0.01,0.1,1,10,100,1000,10000,100000]

    avgsc_lst,avgsc_train_lst,avgsc_hld_lst = [],[],[]
    avgsc,avgsc_train,avgsc_hld = 0,0,0

    i=0
    for train_index, test_index in kf.split(X_train_new):
#         if i>0: break
#         i=i+1
        X_train_cur, X_test_cur = X_train_new[train_index], X_train_new[test_index]
        y_train_cur, y_test_cur = y_train_new[train_index], y_train_new[test_index]
        X_train_train, X_val,y_train_train,y_val = train_test_split(X_train_cur,y_train_cur, test_size=0.25,random_state=Hcurstate)

        print(X_train_train.shape)
        print(X_val.shape)            
        
        bestPerformingModel = Ridge(alpha=1.0,random_state=Hcurstate)
        bestscore = maxintval

        for val in val_arr:
            clf = Ridge(alpha=val,random_state=Hcurstate)
            clf = clf.fit(X_train_train, y_train_train)
            y_pred = clf.predict(X_val)
            sc = sqrt(mean_squared_error(y_pred, y_val))
#             print(sc)
            if bestscore > sc:
                bestscore = sc
                bestPerformingModel = clf


        y_pred = bestPerformingModel.predict(X_train_cur)
        bscr_train = sqrt(mean_squared_error(y_pred, y_train_cur))
        
        y_pred = bestPerformingModel.predict(X_test_cur)
        bscr = sqrt(mean_squared_error(y_pred, y_test_cur))
        
        y_pred = bestPerformingModel.predict(X_test)
        bscr_hld = sqrt(mean_squared_error(y_pred, y_test))

        avgsc_train_lst.append(bscr_train)
        avgsc_lst.append(bscr)
        avgsc_hld_lst.append(bscr_hld)

        avgsc_train = avgsc_train + bscr_train    
        avgsc = avgsc + bscr
        avgsc_hld = avgsc_hld + bscr_hld

        print(bscr_train)
        print(bscr)
        print(bscr_hld)
    

    print('5-fold Train, Validation, and Test loss:')
    print(avgsc_train_lst)
    print(avgsc_lst)
    print(avgsc_hld_lst)
    
    print('Avg Train, Validation, and Test loss:')    
    print(avgsc_train/k)
    print(avgsc/k)
    print(avgsc_hld/k)
    
    return avgsc_train_lst,avgsc_lst,avgsc_hld_lst

def RandForestRegressor(data1,y):
    X_train, X_test,y_train,y_test = train_test_split(data1,y, test_size=0.2,random_state=Hcurstate)

    X_train_new = X_train.reset_index(drop=True)
    y_train_new = y_train.reset_index(drop=True)
    
    X_train_new = X_train_new.values
    y_train_new = y_train_new.values

    k = 5
    kf = KFold(n_splits=k,random_state=Hcurstate)
    avg_train_acc,avg_test_acc = 0,0

    n_estimators_grid = [5,25,50,75,100,500]
    max_depth_grid = [5,10,25,50,100,500]

    avgsc_lst,avgsc_train_lst,avgsc_hld_lst = [],[],[]
    avgsc,avgsc_train,avgsc_hld = 0,0,0

    i=0
    for train_index, test_index in kf.split(X_train_new):
#         if i>0: break
#         i=i+1        
        X_train_cur, X_test_cur = X_train_new[train_index], X_train_new[test_index]
        y_train_cur, y_test_cur = y_train_new[train_index], y_train_new[test_index]
        X_train_train, X_val,y_train_train,y_val = train_test_split(X_train_cur,y_train_cur, test_size=0.25,random_state=Hcurstate)

        print(X_train_train.shape)
        print(X_val.shape)            
        
        bestPerformingModel = RandomForestRegressor(n_estimators=10,max_depth=5, random_state=Hcurstate)
        bestscore = maxintval
        for ne in n_estimators_grid:
            for md in max_depth_grid:
                clf = RandomForestRegressor(n_estimators=ne,max_depth=md, random_state=Hcurstate)
                clf = clf.fit(X_train_train, y_train_train)
                
                y_pred = clf.predict(X_val)
                sc = sqrt(mean_squared_error(y_pred, y_val))                
                sc = clf.score(X_val, y_val)

                if bestscore > sc:
                    bestscore = sc
                    bestPerformingModel = clf

        y_pred = bestPerformingModel.predict(X_train_cur)
        bscr_train = sqrt(mean_squared_error(y_pred, y_train_cur))
        
        y_pred = bestPerformingModel.predict(X_test_cur)
        bscr = sqrt(mean_squared_error(y_pred, y_test_cur))
        
        y_pred = bestPerformingModel.predict(X_test)
        bscr_hld = sqrt(mean_squared_error(y_pred, y_test))

        avgsc_train_lst.append(bscr_train)
        avgsc_lst.append(bscr)
        avgsc_hld_lst.append(bscr_hld)

        avgsc_train = avgsc_train + bscr_train    
        avgsc = avgsc + bscr
        avgsc_hld = avgsc_hld + bscr_hld

        print(bscr_train)
        print(bscr)
        print(bscr_hld)
    
    print('5-fold Train, Validation, and Test loss:')
    print(avgsc_train_lst)
    print(avgsc_lst)
    print(avgsc_hld_lst)
    
    print('Avg Train, Validation, and Test loss:')    
    print(avgsc_train/k)
    print(avgsc/k)
    print(avgsc_hld/k)

    y_pred = bestPerformingModel.predict(X_test)
    
    return avgsc_train_lst,avgsc_lst,avgsc_hld_lst


def MLPRegressorr(data1,y):
    
    X_train, X_test,y_train,y_test = train_test_split(data1,y, test_size=0.2,random_state=Hcurstate)

    X_train_new = X_train.reset_index(drop=True)
    y_train_new = y_train.reset_index(drop=True)
    
    X_train_new = X_train_new.values
    y_train_new = y_train_new.values

    k = 5
    kf = KFold(n_splits=k,random_state=Hcurstate)
    avg_train_acc,avg_test_acc = 0,0

    n_estimators_grid = [5,25,50,75,100,500]
    max_depth_grid = [5,10,25,50,100,500]

    avgsc_lst,avgsc_train_lst,avgsc_hld_lst = [],[],[]
    avgsc,avgsc_train,avgsc_hld = 0,0,0

    i=0
    for train_index, test_index in kf.split(X_train_new):
#         if i>0: break
#         i=i+1        
        X_train_cur, X_test_cur = X_train_new[train_index], X_train_new[test_index]
        y_train_cur, y_test_cur = y_train_new[train_index], y_train_new[test_index]
        X_train_train, X_val,y_train_train,y_val = train_test_split(X_train_cur,y_train_cur, test_size=0.25,random_state=Hcurstate)

        print(X_train_train.shape)
        print(X_val.shape)            
        
        bestPerformingModel = MLPRegressor(hidden_layer_sizes=(100, 100), max_iter=300 , random_state=Hcurstate)
        bestPerformingModel = bestPerformingModel.fit(X_train, y_train)
        print(bestPerformingModel.n_layers_)

        y_pred = bestPerformingModel.predict(X_train_cur)
        bscr_train = sqrt(mean_squared_error(y_pred, y_train_cur))
        
        y_pred = bestPerformingModel.predict(X_test_cur)
        bscr = sqrt(mean_squared_error(y_pred, y_test_cur))
        
        y_pred = bestPerformingModel.predict(X_test)
        bscr_hld = sqrt(mean_squared_error(y_pred, y_test))

        avgsc_train_lst.append(bscr_train)
        avgsc_lst.append(bscr)
        avgsc_hld_lst.append(bscr_hld)

        avgsc_train = avgsc_train + bscr_train    
        avgsc = avgsc + bscr
        avgsc_hld = avgsc_hld + bscr_hld

        print(bscr_train)
        print(bscr)
        print(bscr_hld)
    
    print('5-fold Train, Validation, and Test loss:')
    print(avgsc_train_lst)
    print(avgsc_lst)
    print(avgsc_hld_lst)
    
    print('Avg Train, Validation, and Test loss:')    
    print(avgsc_train/k)
    print(avgsc/k)
    print(avgsc_hld/k)

    y_pred = bestPerformingModel.predict(X_test)
    
    return avgsc_train_lst,avgsc_lst,avgsc_hld_lst


def MLPClassifierr(data1,y):
    
    X_train, X_test,y_train,y_test = train_test_split(data1,y, test_size=0.2,random_state=Hcurstate)

    X_train_new = X_train.reset_index(drop=True)
    y_train_new = y_train.reset_index(drop=True)
    
    X_train_new = X_train_new.values
    y_train_new = y_train_new.values

    k = 5
    kf = KFold(n_splits=k,random_state=Hcurstate)
    avg_train_acc,avg_test_acc = 0,0

    n_estimators_grid = [5,25,50,75,100,500]
    max_depth_grid = [5,10,25,50,100,500]

    avgsc_lst,avgsc_train_lst,avgsc_hld_lst = [],[],[]
    avgsc,avgsc_train,avgsc_hld = 0,0,0

    i=0
    for train_index, test_index in kf.split(X_train_new):
#         if i>0: break
#         i=i+1        
        X_train_cur, X_test_cur = X_train_new[train_index], X_train_new[test_index]
        y_train_cur, y_test_cur = y_train_new[train_index], y_train_new[test_index]
        X_train_train, X_val,y_train_train,y_val = train_test_split(X_train_cur,y_train_cur, test_size=0.25,random_state=Hcurstate)

        print(X_train_train.shape)
        print(X_val.shape)            
        
        bestPerformingModel = MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=300 , random_state=Hcurstate)
        bestPerformingModel = bestPerformingModel.fit(X_train, y_train)
        
        bscr_train = bestPerformingModel.score(X_train_cur, y_train_cur)
        bscr = bestPerformingModel.score(X_test_cur, y_test_cur)
        bscr_hld = bestPerformingModel.score(X_test, y_test)

        avgsc_train_lst.append(bscr_train)
        avgsc_lst.append(bscr)
        avgsc_hld_lst.append(bscr_hld)

        avgsc_train = avgsc_train + bscr_train    
        avgsc = avgsc + bscr
        avgsc_hld = avgsc_hld + bscr_hld

        print(bscr_train)
        print(bscr)
        print(bscr_hld)
    
    print('5-fold Train, Validation, and Test Accuracies:')
    print(avgsc_train_lst)
    print(avgsc_lst)
    print(avgsc_hld_lst)
    
    print('Avg Train, Validation, and Test Accuracies:')    
    print(avgsc_train/k)
    print(avgsc/k)
    print(avgsc_hld/k)

    y_pred = bestPerformingModel.predict(X_test)
    
    return avgsc_train_lst,avgsc_lst,avgsc_hld_lst
